<div style="color:red;background-color:black">
Diamond Light Source
<br style="color:red;background-color:antiquewhite"><h1>Python Language: Exception Handling</h1>  

©2000-20 Chris Seddon 
</div>

## 1
Exception handling is the modern way of handling errors in programs.  Almost all programming languages use exception handling; C being the notable 'exception'.  Exception handling is split into 3 parts:  

* try block: defining a region of code to monitor for exceptions
* raise statement: what to do when a problem arises
* except block: block of code to handle or respond to the exception

Here is a simple example:

In [ ]:
try:
    for n in range(5, -5, -1):
        print(10/n)
except:
    print("division failed")

## 2 
You'll notice that there was no "raise" statement evident in the above example.  This was because the division by zero was detected by the Python runtime and the exception was raised there.  We can't see the code for the runtime, but be reassured an exception was raised.

In the following example we raise the exception ourselves, rather than relying on the Python runtime.

In [ ]:
try:
    for n in range(5, -5, -1):
        if n == 0: 
            raise Exception()
        print(10/n)
except:
    print("division failed")

## 3
The mechanics of exception handling involve creating an exception object when a problem is detected.  In the above example, when "n" is zero, we know the code is about to fail, so we create the exception object from the "Exception" class with the line: <pre>
raise Exception()
</pre>
The "Exception" class is the base class for all exceptions.  As we will see later, we often raise exceptions using exception objects from derived classes.  You can find a full list of built-in exception classes in the Python documentation:
<a href="https://docs.python.org/3/library/exceptions.html">built-in exceptions</a>

The example above can be rewriiten using objects of the "ZeroDivisionError" class:

In [ ]:
try:
    for n in range(5, -5, -1):
        if n == 0: 
            raise ZeroDivisionError()
        print(10/n)
except ZeroDivisionError:
    print("division failed")

## 4
It is often a good idea to define our own exception classes.  When we define our own exception class we should derive from "Exception", but we don't need to add any methods or attributes.  
Such classes are called "tag" classes; they don't add any functionality to the "Exception" class, but they do have a distinct name (or tag): <pre>class TooBig(Exception): pass</pre>

Code using our own tag class is demonstrated below:

In [ ]:
class TooBig(Exception): pass

try:
    for x in range(25, 200, 25):
        if x > 100: 
            raise TooBig()
        else:
            print(x)
except TooBig as e:
    print(f"{x} is too big")    

## 5
There is really no need to add an error message when we raise a "TooBig" exception; the name says it all.  Note that a TooBig object is raised in the try block and subsequently caught in the except block.  When the object is caught we need to give it a name: <pre>except TooBig as e:</pre>  

Try blocks can appear in functions - this adds flexibility to our code.  The following example uses functions and another user defined tag class:

In [ ]:
class TooBig(Exception): pass
class MuchTooBig(Exception): pass

def checkValue(x):
    try:
        if x > 200: 
            raise MuchTooBig()
        elif x > 100:
            raise TooBig()
        else:
            print(x)
    except MuchTooBig as e:
        print(f"{x} is much too big")
    except TooBig as e:
        print(f"{x} is too big")

checkValue(50)
checkValue(150)
checkValue(250)

## 6
An important consideration with exception handling is to realise that it is a "GOTO" technology.  What I mean by this is that as soon as an exception is raised, the remaining statements in the try block are bypassed and we "GOTO" the except block directly.

To demonstrate this we can write:

In [ ]:
try:
    print("before exception")
    print("before exception")
    print("before exception")
    raise Exception()
    print("after exception")   # these 3 statements are never executed
    print("after exception")
    print("after exception")
except:
    print("in the except block")
    print("in the except block")
    
print("outside the try block")

## 7
In the example above, the 3 print statements: <pre>print("after exception")</pre> are bypassed as soon as the exception is raised and control is passed to the except block.  Once the except block is completed, execution continues with the first statement after the except block.  

Exception handling provides additional facilities to the above.  One such feature is to branch our code depending on whether an exception occurs or not: 
 
* except: if exception occurs 
* else:   if no exception  

Consider the following:

In [ ]:
from math import sqrt

def squareRoot(x):
    try:
        root = sqrt(x)
    except Exception as e:
        print("sqrt() failed ...")
        print(e)
    else:
        print("sqrt() succeeded ...")
        print(root)

squareRoot(100)
squareRoot(-100)

## 8
Another feature we can use is the "finally" block.  Code in the "finally" block is guaranteed to be executed whether or not an exception is raised.  Finally blocks are used to perform operations that should be completed, even if an exception is raised.  

For example, finally blocks can close files, release locks, close database connections etc.

Here is an example:

In [ ]:
class TooBig(Exception): pass
class MuchTooBig(Exception): pass

def checkValue(x):
    try:
        if x > 200: 
            raise MuchTooBig()
        elif x > 100:
            raise TooBig()
        else:
            print(x)
    except MuchTooBig as e:
        print(f"{x} is much too big")
    except TooBig as e:
        print(f"{x} is too big")
    finally:
        print("finally block is always called ...")

checkValue(50)
checkValue(150)
checkValue(250)

## 9
So far we have only looked at exceptions being raised and caught in the same function.  Often an exception is raised in one function and caught in another.  To make this work, realise that a try block extends to code in all functions executed as part of the try block and not just within the function.

Here is an example ...

In [ ]:
class Point:
    def __init__(self, x0, y0):
        print("CTOR")
        self.x = x0
        self.y = y0
        
    def __del__(self):
        print("DTOR")
        
def f1():
    print("Entering f1")
    p1 = Point(50, 17)
    f2()
    print("Leaving f1")

def f2():
    print("Entering f2")
    p2 = Point(70, 83)
    f3()
    print("Leaving f2")
    
def f3():
    print("Entering f3")
    p3 = Point(29, 43)
    raise Exception("Some exception")
    print("Leaving f3")
    
def main():
    try:
        f1()
    except Exception as e:
        print(e)

main()
print("End of program")

## 10
The example above illustrates the fact that the try block defined in "main" extends to code in "f1", "f2" and "f3".  When an exception is raised in "f3", control transfers immediately to the except block in "main".  This means the three functions "f1", "f2" and "f3" are terminated abruptly and the statements: <pre>
print("Leaving f1")
print("Leaving f2")
print("Leaving f3")
</pre>
are never executed.  As I said above, exception handling is a "GOTO" technology.  

Note the "CTOR" and "DTOR" print statements.  The "CTOR" messages are displayed when the "Point" objects are created.  The "DTOR" messages are displayed when Python's garbage collector removes the object from memory.  Note that garbage collectors in some Python interpreters may not clean up objects immediately.  It's even possible that the program finishes before the garbage collector kicks in and the "DTOR" is never called.  

You should be aware that just printing error messages in exception handlers is considered poor practice.  This is because programs are often run automatically and it is very easy to miss messages printed on the console.  Instead, it is recommended to use logging to record messages.  Here is an example of doing just that:

In [ ]:
import logging, os

# create log file
if not os.path.exists('logs'): os.mkdir('logs')
logging.basicConfig(filename='logs/exception.log',level=logging.ERROR)


def do_work():
    # if an exception is encountered in library code we might not know
    # what to do about it, so log error and rethrow
    logging.error('problem in library code ...')
    raise Exception('do_work() failed')

def check():
    try:
        # perform some work in library code
        do_work()
    except Exception as e:
        # handle the error in your code
        # because you know what you want to do about it
        print("handling exception generated by library code ...")

check()
check()
check()

## 11
Let's look at the log file:

In [ ]:
%%bash
cat logs/exception.log

## 12
Finally, we will conclude our short tour of exception handling by looking at assert statements.  

When we write a function we often assume that the function is called with sensible parameters and we don't make any checks.  Using assert statements, we can verify if pre and post conditions for calling functions are met.  If these conditions are not met then the function will not work as expected and the assert statements will raise an exception.  
Normally we do not provide except blocks for assert statements.  The pre or post conditions should always be met, otherwise the program is invalid and we let it terminate in error as in the following example:

In [ ]:
def CalculateQuartile(percent):
    assert percent >= 0 and percent <= 100
    quartile = 1
    if percent > 25:
        quartile = 2
    if percent > 50:
        quartile = 3
    if percent > 75:
        quartile = 4
    print(f"{percent} is in quartile {quartile}")

CalculateQuartile(3)
CalculateQuartile(34)
CalculateQuartile(68)
CalculateQuartile(93)
CalculateQuartile(104)

## 13
However, if you want to provide except blocks for such errors without termination the program it's best to provide a user defined exception instead.  The exception raised by an assert statement is designed to be displayed in a stack trace and not in an except block.

Using a user defined exception class we could then recode the above as:

In [ ]:
class InvalidPreCondition(Exception): pass

def CalculateQuartile(percent):
    if percent < 0 or percent > 100:
        raise InvalidPreCondition()
    quartile = 1
    if percent > 25:
        quartile = 2
    if percent > 50:
        quartile = 3
    if percent > 75:
        quartile = 4
    print(f"{percent} is in quartile {quartile}")

try:
    CalculateQuartile(3)
    CalculateQuartile(34)
    CalculateQuartile(68)
    CalculateQuartile(93)
    CalculateQuartile(104)
except InvalidPreCondition:
    print("pre-condition failed")